In [ ]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Read in the MD trajectory

Below we read in the production simulation trajectory using the topology available in the PDB file of the starting structure.

In [ ]:
traj = md.load('ethane_sim.dcd', top='ethane.pdb')
atoms, bonds = traj.topology.to_dataframe()
atoms

## Analyzing the H-C-C-H torsion

In [ ]:
phi_indices = [1, 0, 4, 5] # atoms to define the torsion angle
phi = md.compute_dihedrals(traj, [phi_indices])

phicounts, binedges, otherstuff = plt.hist(phi, bins=90) # create a histogram with 90 bins
plt.title('H-C-C-H torsion angle')
plt.xlabel(r'$\phi$ (rad)')
plt.ylabel('Counts')
plt.show()

In [ ]:
kB = 8.31446/1000 # Boltzmann constant in kcal/mol
Temp = 323.15 # simulation temperature
phicounts[phicounts==0] = 0.1 # get rid of any bins with 0 counts/infinite energy
pmf = -kB*Temp*np.log(phicounts)
pmf = pmf - np.min(pmf) # subtract off minimum value so that energies start from 0

bincenters = (binedges[1:] + binedges[:-1])/2 # compute centers of histogram bins

plt.plot(bincenters, pmf)
plt.title('H-C-C-H torsion pmf')
plt.xlabel(r'$\phi$ (rad)')
plt.ylabel('Relative free energy (kJ/mol)')
plt.show()

The pmf looks a bit jagged (mainly due to our finite sampling), so let's try to smooth it out using the LOWESS (locally weighted scatterplot smoothing) algorithm:

In [ ]:
import statsmodels.api as sm

pmf_smoothed = sm.nonparametric.lowess(pmf, bincenters, frac=0.1)
plt.plot(pmf_smoothed[:,0], pmf_smoothed[:,1])
plt.title('H-C-C-H torsion pmf')
plt.xlabel(r'$\phi$ (rad)')
plt.ylabel('Relative free energy (kJ/mol)')
plt.show()

## Analyzing the C-C bond length

In [ ]:
bond_indices = [0, 4] # atoms to define the bond length
bondlength = md.compute_distances(traj, [bond_indices])

bondcounts, binedges, otherstuff = plt.hist(bondlength, bins=100)
plt.title('C-C bond length')
plt.xlabel('Bond length (nm)')
plt.ylabel('Counts')
plt.show()

In [ ]:
bondcounts[bondcounts==0] = 0.1
pmf = -kB*Temp*np.log(bondcounts)
pmf = pmf - np.min(pmf)

bincenters = (binedges[1:] + binedges[:-1])/2

pmf_smoothed = sm.nonparametric.lowess(pmf, bincenters, frac=0.1)
pmf_s = pmf_smoothed[:,1] - np.min(pmf_smoothed[:,1])

plt.plot(bincenters, pmf_s)
plt.xlabel('Bond length (nm)')
plt.ylabel('Relative free energy (kJ/mol)')
plt.show()